In [1]:
import numpy as np
import scipy

In [4]:
input_data = np.genfromtxt('input1.1.csv', delimiter=',')
m, n = input_data.shape[0] - 2, input_data[1] - 1
z = input_data[0, :-1]
c = input_data[1, :-1]
b = input_data[2:, -1]
A = input_data[2:, :-1]
print("Initial Feasible Point (z):", z)
print("Cost Vector (c):", c)
print("Constraint Vector (b):", b)
print("Matrix A:")
print(A)

print(A.shape, b.shape, c.shape, z.shape)
if np.any(A @ z > b):
    print("z does not belong to the solution set")

Initial Feasible Point (z): [1. 0.]
Cost Vector (c): [2. 5.]
Constraint Vector (b): [4. 9. 3. 0. 0.]
Matrix A:
[[ 2. -1.]
 [ 1.  2.]
 [-1.  1.]
 [-1.  0.]
 [ 0. -1.]]
(5, 2) (5,) (2,) (2,)


In [6]:
def find_optimal_vertex(A, b, c, z):
    while True:
        tight_rows = np.isclose(A @ z, b)
        num_tight_rows = np.count_nonzero(tight_rows)
        if num_tight_rows == 0:
            u = np.ones_like(z)
        elif num_tight_rows == A.shape[1]:
            break
        else:
            A1 = A[tight_rows]
            A2 = A[~tight_rows]
            u = scipy.linalg.null_space(A1).T[0]
        low = 0
        high = 1e2
        alpha = 0
        while high - low > 1e-15:
            alpha = (low + high) / 2
            z_new = z + (alpha * u)
            new_tight_rows = np.isclose(A @ z_new, b)
            if not np.any(A @ z_new > b) or np.count_nonzero(new_tight_rows) < num_tight_rows:
                low = alpha
            else:
                high = alpha
            alpha = low
        z = z + (alpha * u)
    print("Initial vertex: {}".format(z))

    while True:
        print("Now at vertex: {}".format(z))
        print("Value of objective function at this vertex = {}".format(np.dot(z, c)))
        tight_rows = np.isclose(A @ z, b)
        A1 = A[tight_rows]
        A1_inv = np.linalg.inv(A1)
        alphas = A1_inv.T @ c
        if np.all(alphas > 0):
            break
        negative_index = np.where(alphas < 0)[0][0]
        column = A1_inv[:, negative_index]

        low = 0
        high = 1e2
        beta = 0
        while high - low > 1e-15:
            beta = (low + high) / 2
            z_new = z - (beta * column)
            if np.any(A @ z_new > b):
                high = beta
            else:
                low = beta
            beta = low
        z = z - (beta * column)
    return z

optimal = find_optimal_vertex(A, b, c, z)
print(optimal)

Initial vertex: [2. 0.]
Now at vertex: [2. 0.]
Value of objective function at this vertex = 4.0
Now at vertex: [3.4 2.8]
Value of objective function at this vertex = 20.8
Now at vertex: [1. 4.]
Value of objective function at this vertex = 22.0
[1. 4.]
